Code based on:
Bowles, M. (2019). Machine Learning with Spark and Python: Essential Techniques for Predictive Analytics. John Wiley & Sons.

In [2]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.7 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 5.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845509 sha256=3ed804ac019dbde596f046960811758b1a716370a93effb3841f44def1adcea0
  Stored in directory: /home/vagrant/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler, OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
spark = SparkSession.builder.appName("abalone").getOrCreate()

In [ ]:
from pyspark.sql.types import StructType,StructField,IntegerType,DoubleType,\
                              StringType
aba_schema = StructType([\
    StructField("Sex",StringType(),True),
    StructField("Length",DoubleType(),True),
    StructField("Diameter",DoubleType(),True),
    StructField("Height",DoubleType(),True),
    StructField("Whole",DoubleType(),True),
    StructField("Shucked wt",DoubleType(),True),
    StructField("Viscera wt",DoubleType(),True),
    StructField("Shell wt",DoubleType(),True),
    StructField("Rings",IntegerType(),True)])
aba = spark.read.format("csv").option("header", "False").schema(aba_schema)\
      .option("inferSchema", "True").load("Data/abalone/abalone.data")

In [ ]:
aba.show(3)

In [ ]:
num_cols = aba.columns[1:-1]

In [ ]:
string_indexer = StringIndexer(inputCol="Sex", outputCol="SexIdx")

In [ ]:
encoder = OneHotEncoder(inputCols=[string_indexer.getOutputCol()], outputCols=["SexVec"])

In [ ]:
inputs = ["SexVec"] + num_cols

In [ ]:
vec_assembler = VectorAssembler(inputCols=inputs,outputCol="features")

In [ ]:
stages = [string_indexer, encoder, vec_assembler]

In [ ]:
pipeline = Pipeline(stages = stages)
pipeline_mod = pipeline.fit(aba)
aba_transfd = pipeline_mod.transform(aba)
inp_aba = aba_transfd.select(['features', 'Rings'])

In [ ]:
inp_aba.show(4)

In [ ]:
train_aba, test_aba = inp_aba.randomSplit([0.7, 0.3], seed = 1234)

In [ ]:
regr_aba = LinearRegression(featuresCol = 'features', labelCol='Rings', maxIter=10, regParam=0.003, elasticNetParam=0.8)

In [ ]:
regr_mod_aba = regr_aba.fit(train_aba)
print("Weights and intercept: ", regr_mod_aba.coefficients, regr_mod_aba.intercept)

In [ ]:
train_aba_summary = regr_mod_aba.summary
print("RMSE (training): ", train_aba_summary.rootMeanSquaredError)
print("R Squared (training):", train_aba_summary.r2)

In [ ]:
regr_aba_preds = regr_mod_aba.transform(test_aba)
regr_aba_preds.select("prediction","Rings","features").show(8)

In [ ]:
regr_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Rings",metricName="r2")
print("R Squared (test):", regr_eval.evaluate(regr_aba_preds))